<center><h1>Informarion Retrival Project</h1></center>
<center><h1>Build an Intelligent Information Retrival System</h1></center>

## Importation des librairies et du dataset

In [1]:
import nltk
import numpy as np
import pandas as pd
import re
import json
import warnings
from metric_tools import compute_metrics
warnings.filterwarnings("ignore")

In [2]:
fichier_inverse = pd.read_csv("../fichier_inverse.csv")
fichier_inverse = fichier_inverse.drop("Unnamed: 0", axis=1)
fichier_inverse

,Word,Document,Frequence,Poid
0,0,D443,1,0.210977
1,0.18,D1090,1,0.527442
2,0.5,D720,1,0.791163
3,0.7%,D691,1,0.395581
4,000,D687,2,1.071347
...,...,...,...,...
77177,zipfian,D329,1,0.527442
77178,zone,D62,2,0.575391
77179,zoolog,D755,1,0.632930
77180,zuckerman,D1291,1,1.431959


In [3]:
info_queries = pd.read_csv("../info_queries.csv")
info_queries = info_queries.drop("Unnamed: 0", axis=1)
info_queries

,Query,Word
0,Q1,problem
1,Q1,concern
2,Q1,make
3,Q1,descript
4,Q1,titl
...,...,...
5188,Q112,algorithm
5189,Q112,compar
5190,Q112,previous
5191,Q112,describ


In [4]:
eval_df = pd.read_csv("../evaluation.csv")
eval_df = eval_df.drop("Unnamed: 0", axis=1)
eval_df

,Query,Document
0,Q1,D28
1,Q1,D35
2,Q1,D38
3,Q1,D42
4,Q1,D43
5,Q1,D52
6,Q1,D65
7,Q1,D76
8,Q1,D86
9,Q1,D150


### SRI basé sur le modèle probabiliste avec la fonction BM25

In [6]:
# SRI Probabiliste en utilisant la fonction BM25
def bm25(query, fichier_inverse, info_queries, B, K):
    N = len(fichier_inverse['Document'].unique()) # Nombre de documents

    words = list(info_queries.loc[info_queries["Query"] == query, "Word"]) # Liste des mots de la requête
    df = fichier_inverse.loc[fichier_inverse["Word"].isin(words)] # Fichier inverse contenant les mots de la requête

    nb_doc = df.groupby('Word')['Document'].count().to_frame() # Nombre de documents contenant les mots de la requête
    nb_doc.reset_index(inplace=True) # Réinitialisation de l'index
    nb_doc.rename(columns = {'Document':'Nombre document contenu'}, inplace = True) # Renommage de la colonne
    df = df.merge(nb_doc, on="Word") # Fusion des deux dataframes

    nb_termes_doc = fichier_inverse.groupby('Document')['Frequence'].sum().to_frame() # Nombre de termes par document
    nb_termes_doc.reset_index(inplace=True) # Réinitialisation de l'index
    nb_termes_doc.rename(columns = {'Frequence':'dl'}, inplace = True) # Renommage de la colonne
    df = df.merge(nb_termes_doc, on="Document") # Fusion des deux dataframes
    
    df["avdl"]= nb_termes_doc["dl"].mean() # Moyenne du nombre de termes par document
    
    rsv_list = [] # Liste des RSV

    for d in df["Document"].unique(): # Pour chaque document
        temp = df.loc[df["Document"] == d, ["Frequence", "Nombre document contenu", "dl", "avdl"]] # Dataframe temporaire
        ni = temp["Nombre document contenu"] # Nombre de documents contenant le mot

        # Il est 3h du mat flemme d'expliquer la formule regarde le pdf tu vas comprendre
        rsv = np.multiply(np.divide(temp["Frequence"],
                            np.add(np.multiply(K, 
                            np.add(np.subtract(1, B), 
                                    np.multiply(B, np.divide(temp["dl"], temp["avdl"])))), 
                                                temp["Frequence"])), 
                                                np.log10(np.divide(np.add(
                                                        np.subtract(N, ni), 0.5), np.add(ni, 0.5))))
        
        rsv_list.append([d, np.sum(rsv)]) # Ajout du RSV dans la liste

    df = pd.DataFrame(rsv_list, columns = ["Document","RSV"]) # Création d'un dataframe avec la liste des RSV
    df = df.sort_values(by=["RSV"]) # Tri des RSV par ordre croissant

    return df.reindex(index=df.index[::-1]) # Retourne le dataframe trié par ordre décroissant

bm25("Q1", fichier_inverse, info_queries, 0.6, 1.5)

,Document,RSV
77,D429,2.420663
163,D42,2.395005
54,D447,2.377216
257,D1419,2.347758
212,D489,2.318881
...,...,...
763,D593,0.182191
586,D400,0.180956
814,D158,0.180015
808,D248,0.180015


In [7]:
# Calcule des métriques
metrics = []
for query in info_queries["Query"].unique():
    bm25_result = bm25(query, fichier_inverse, info_queries, 0.6, 1.5)  
    p, p5, p10, r, f = compute_metrics(query, eval_df, bm25_result)
    metrics.append([query, p, p5, p10, r, f])
metrics = pd.DataFrame(metrics, columns=["Query", "Precision", "P@5", "P@10", "Rappel", "F-Mesure"])
metrics

,Query,Precision,P@5,P@10,Rappel,F-Mesure
0,Q1,0.044444,0.4,0.3,1.000000,0.085106
1,Q2,0.023901,0.2,0.1,0.961538,0.046642
2,Q3,0.048423,0.8,0.4,0.977273,0.092275
3,Q4,0.015066,0.4,0.2,1.000000,0.029685
4,Q5,0.017002,0.2,0.1,0.916667,0.033384
...,...,...,...,...,...,...
107,Q108,0.000000,0.0,0.0,0.000000,0.000000
108,Q109,0.048917,0.6,0.6,0.985915,0.093209
109,Q110,0.000000,0.0,0.0,0.000000,0.000000
110,Q111,0.004651,0.2,0.2,1.000000,0.009259


In [9]:
metrics.to_csv("../csv_docs/bm25_metrics.csv", index=False)